# Vos noms

Renseigner vos noms dans la cellule ci-dessous :

# TP NetworkX

Ce TP, a pour but la manipulation de graphes en Python, en mémoire, avec la librairie **[NetworkX](https://networkx.org/documentation/stable/index.html)**.

In [1]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


## Contexte

Nous allons analyser des **interactions entre objets connectés** dans les usines de Transpomme. Les objets sont de 3 types :
- des **périphériques de capture** PomSort au plus près des machines (caméra + modèles d'IA embarqués en mode "edge computing"). Il y en a plus de 1000
- des **concentrateurs** qui reçoivent les données PomSort au niveau de chaque atelier. Il y en a quelques centaines
- des **super-concentrateurs** qui agrègent les données venant des concentrateurs d'atelier, pour les consolider au niveau usine. Il y en a quelque-uns

Les périphériques de capture parlent aux concentrateurs (flèches **(1)**), les concentrateurs aux super-concentrateurs (flèches **(2)**), et parfois à d'autres concentrateurs de la même usine (flèche **(3)**), et les super-concentrateurs parlent entre eux (flèche **(4)**).

<img src="https://github.com/tvial/BUT-R6.VCOD.05-TP-NetworkX/blob/main/graph.png?raw=true">

Il faut aussi savoir que nos données couvrent 2 pays : la France, et la Suisse. La France a plus d'usines que la Suisse, et il n'y a pas d'échanges de données entres les usines de ces pays.

Dans notre modélisation, chaque périphérique, concentrateur ou super-concentrateur est un noeud de graphe, et les interactions sont les arcs. Le graphe sera dirigé, dans la mesure où les interactions ont un sens (_"A envoie des données à B"_).

## Données

Les interactions sont données dans un fichier CSV, `interactions.csv`. C'est une liste d'adjacence :
```
timestamp,from_device,to_device
2025-03-08 15:23:07,108.175.212.176,108.172.90.112
2025-02-25 12:03:07,108.171.174.176,108.170.255.48
2025-02-20 00:39:42,108.172.47.176,108.171.130.112
2025-02-10 01:09:52,108.171.149.48,108.170.254.176

...
```

- `timestamp` : date & heure de l'interaction
- `from_device` : adresse IP du noeud _A_qui initie l'interaction
- `to_device` : adresse IP du noeud _B_ qui reçoit l'interaction

# Questions

## 1. Chargement et mise en forme des données

Charger le fichier d'interactions dans un dataframe Pandas nommé `interactions`, et l'agréger pour obtenir les colonnes suivantes :
- `from_device`
- `to_device`
- `n_interactions` : nombre d'interactions entre les 2 noeuds (on n'est pas intéressé par les timestamps en tant que tels, juste leur nombre)

Contrôle :

In [2]:
interactions.sort_values(['from_device', 'to_device']).head()

,from_device,to_device,n
0,108.170.254.176,108.170.255.176,260
1,108.170.254.48,108.170.255.176,233
2,108.170.255.176,108.171.128.240,336
3,108.170.255.176,108.171.128.48,196
4,108.170.255.48,108.170.255.176,300


## 2. Création du graphe

Créer un graphe NetworkX dirigé (`DiGraph`), nommé `graph`, et l'alimenter avec les arcs donnés par les interactions. On peut se contenter de créer uniquement les arcs, les noeuds seront ajoutés automatiquement si nécessaire.

Les arcs doivent être dirigés dans le bon sens, et avoir un attribut `n`, qui contient la valeur de la colonne de même nom dans le dataframe.

Contrôle du nombre de noeuds et d'arcs :

In [297]:
print('Noeuds :', len(graph.nodes), '/ arcs :', len(graph.edges))

Noeuds : 1531 / arcs : 1591


## 3. Visualisation

Calculer des positions de noeuds pour le layout "_spring_". Régler le paramètre `k` à 0.06.

Représenter ensuite le graphe avec les positions calculées précédemment, et interpréter sommairement sa forme.

Quelques recommandations :
- la taille des noeuds par défaut est trop élevée, changer pour une taille de 10
- avec le chevauchement on ne voit pas grand-chose, vous pouvez régler la transparence à 0.5
- si le résultat reste difficile à interpréter visuellement, il peut être nécessaire de recalculer les positions 2 ou 3 fois jusqu'à obtenir une représentation exploitable. Cela change les conditions initiales aléatoires du calcul

## 4. Distribution des degrés

Représenter la distribution statistique des degrés **entrants** des noeuds (sans tenir compte de la valeur de `n`). Cette distribution doit prendre la forme d'un histogramme ; choisir un nombre de "bins" approprié pour faire ressortir la structure. Comment peut-on interpréter la distribution avec ce que l'on sait du graphe ?

## 5. Classification des noeuds

A partir de la question précédente, trouver une règle qui permette d'attribuer à chaque noeud son type (périphérique de capture, concentrateur, super-concentrateur).

Combien y a-t-il de noeuds de chaque type ?

Ajouter, à chaque noeud du graphe, un attribut `type` qui a pour valeur `capture`, `concentrator` ou `super-concentrator` selon son type.

## 6. Amélioration de la dataviz

Reprendre la dataviz de la question 3, pour changer la représentation des noeuds :
- capture : taille 8, couleur `tab:blue`
- concentrateurs : taille 30, couleur `tab:orange`
- super-concentrateurs : taille 100, couleur `tab:green`

Vous pourrez réutiliser les positions de noeuds précédentes, sans les recalculer.

## 7. Identification des pays

Analyser le graphe pour retrouver la distinction entre les 2 pays (France, Suisse). Quelles sont les adresses IP des super-concentrateurs de chaque pays ?

NB : l'algorithme de positionnement des noeuds peut ne pas faire apparaître la séparation dans la dataviz. Cela arrive quand les groupes de noeuds se chevauchent.

## 8. Calcul de centralité v1

Calculer la centralité "betweenness" non normalisée de tous les noeuds du graphe, et afficher les 15 noeuds les plus centraux avec leur mesure de centralité.

Parmi ce top 20, quels super-concentrateurs retrouvez-vous ?

On devrait s'attendre à trouver les super-concentrateurs en premier, car c'est par eux que passent la plupart des chemins. Expliquez pourquoi certains n'apparaissent pas (vous pouvez consulter leur centralité pour avoir un indice). Comment modifier le calcul de centralité pour les faire apparaître ?

## 9. Calcul de centralité v2

Les calculs précédents n'ont pas exploité l'attribut `n` des arcs, qui mesurent combien d'interactions ont eu lieu entre 2 noeuds sur la période d'observation.

Les calculs de centralité peuvent exploiter un poids par arc, via le paramètre `weight`, interprété comme une distance dans la détermination des plus courts chemins.

Si on considère que 2 noeuds sont plus "proches" quand ils ont un grand nombre d'interactions, ajouter un attribut `dist` aux noeuds, qui est l'inverse de leur paramètre `n`. Refaire les calculs de centralité et de "top 15" en pondérant les arcs par cet attribut.

Vous devez constater que les changements sont très peu significatifs : ordre de grandeur, classement du top 15. Avec ce que vous savez du graphe et de la centralité "betwenness", comment expliquer cela ?

## 10. Identification de chemins

Enumérer tous les chemins allant du noeud `108.173.172.48` au noeud `108.170.255.176`, avec les noeuds du parcours. Interpréter le résultat (nombre de chemins).

Afficher les attributs des noeuds du parcours de chaque chemin, et interpréter le résultat.

## 11. Extraction et fusion de sous-graphes

Un virus informatique a été repéré sur le réseau Transpomme ; on sait qu'il a contaminé les périphériques de capture `108.176.176.112` et `108.176.200.240`. On cherche à savoir quels autres noeuds du réseau il a pu infecter, en suivant les envois de données.

Extraire les sous-arbres partant de ces 2 noeuds sources, et les représenter chacun de son côté (en réutilisant les positions, tailles de noeuds et couleurs déjà utilisées plus haut, et cette fois en affichant les noms des noeuds).

Ensuite, fusionner les 2 sous-graphes et représenter le résultat, pour avoir une vue synthétique de l'épidémie.

NB : en faisant l'extraction des sous-graphes, les attributs des noeuds ne sont pas repris, mais leurs noms le sont. Pour récupérer les types des noeuds (nécessaires pour la taille et la couleur), vous pouvez interroger le graphe principal.